In [14]:
from pandas_datareader import data
import mplfinance as mpf
import pandas

import tkinter as tk
import tkinter.ttk as ttk
import datetime as dt
import time
import csv
import os
import sys
from PIL import Image

from tkcalendar import Calendar


#数値を画面に出力するための関数
csvfile_path = './data/stocklist.csv'
df = pandas.read_csv(csvfile_path)

def draw_graph():
    #テキストボックスの内容を取得
    global number, calender_date
    number2=str(number.get())+".JP"
    start = calender_date.get_date().split("/")
    start = '20'+start[2]+'-'+start[0]+'-'+start[1]
    end = dt.datetime.now().strftime('%Y-%m-%d')
    filename = number2+'.png'
    #株式データの取得
    df = data.DataReader(number2, 'stooq', start, end)
    df = df.sort_index()

    #MACDの計算
    exp12 = df['Close'].ewm(span=12, adjust=False).mean()
    exp26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp12 - exp26
    # シグナル計算
    df['Signal'] = df['MACD'].rolling(window=9).mean()
    # ヒストグラム(MACD - シグナル)
    df['Hist'] = df['MACD'] - df['Signal']
    #一目均衡表 9、26、52日間のMAX、MIN値の計算

    max_9 = df['High'].rolling(window=9, min_periods=1).max()
    min_9 = df['Low'].rolling(window=9, min_periods=1).min()

    max_26 = df['High'].rolling(window=26, min_periods=1).max()
    min_26 = df['Low'].rolling(window=26, min_periods=1).min()

    max_52 = df['High'].rolling(window=52).max()
    min_52 = df['Low'].rolling(window=52).min()

    # ５本線の計算

    df['tenkan'] = (max_9 + min_9)/2
    df['base'] = (max_26 + min_26)/2
    df['senkou1'] = ((df['tenkan'] + df['base'])/2).iloc[:-26]
    df['senkou2'] = ((max_52 + min_52)/2).iloc[:-26]
    df['chikou'] = df['Close'].iloc[:-26]

    #RSI(相対力指数)
    # 終値の差分

    df_diff = df['Close'].diff()

    # 値上がり幅と値下がり幅

    df_up, df_down = df_diff.copy(), df_diff.copy()
    df_up[df_up < 0] = 0
    df_down[df_down > 0] = 0
    df_down = df_down * -1

    # 14日間の単純移動平均

    sim14_up = df_up.rolling(window=14).mean()
    sim14_down = df_down.rolling(window=14).mean()

    # RSI
    df['RSI'] = sim14_up / (sim14_up + sim14_down) * 100

    add_plot = [mpf.make_addplot(df['MACD'], color='m', panel=1, secondary_y=False),
            mpf.make_addplot(df['Signal'], color='c', panel=1, secondary_y=False),
            mpf.make_addplot(df['Hist'], type='bar', color='g', panel=1, secondary_y=True)
            mpf.make_addplot(df['base'], color='black'),
            mpf.make_addplot(df['tenkan'], color='red'),
            mpf.make_addplot(df['chikou'], color='darkorange'),
            mpf.make_addplot(df['RSI'], panel=2),
            ]


    #グラフの描画と保存
    mpf.plot(df, title='\n'+number2, type='candle', mav=(5, 25), volume=True, savefig=filename)
 
    #画像のサイズ取得
    img = Image.open(filename)
    width = img.width
    height = img.height
 
    #サブウィンドウの追加
    sub = tk.Toplevel()
    sub.title(number2)
    sub.geometry(str(width)+"x"+str(height))
    #グラフの描画
    graph = tk.PhotoImage(file=filename)
    canvas = tk.Canvas(sub,bg='white', width = width, height = height)
    canvas.place(x=0, y=0)
    canvas.create_image(0,0,image=graph, anchor=tk.NW)

#数値を画面に出力するための関数
def search_table():
    global name, df
    name2=str(name.get())
    df_part = df[df['銘柄名'].str.contains(name2)]
    df_part_str=df_part.astype(str)
    #サブウィンドウの追加
    sub = tk.Toplevel()
    sub.title("search")
    sub.geometry("500x500")
    #グラフの描画
    search_label = ttk.Label(
        sub,
        text=df_part_str,
        width=50,
        relief=tk.SUNKEN,
    )
    search_label.grid(row=0, column=0, padx=20, pady=5)

# トップレベルウィンドウ作成
root = tk.Tk()
root.title("search_and_drawgraph_system")

name_label = ttk.Label(
    root,
    text="name",
    width=5,
    relief=tk.SUNKEN,
)
name_label.grid(row=0, column=0, padx=20, pady=5)

name = tk.Entry(root,width=20)
name.grid(row=0, column=1, padx=10, pady=5)

get = tk.StringVar(root)
get.set("GET!")
get_button = tk.Button(root, textvariable=get, command=search_table)
get_button.grid(row=0, column=3, padx=10, pady=5)
    

number_label = ttk.Label(
    root,
    text="num",
    width=5,
    relief=tk.SUNKEN,
)
number_label.config(background='white')
number_label.grid(row=1, column=0, padx=20, pady=5)

number = tk.Entry(width=20)
number.grid(row=1, column=1, padx=10, pady=5)

date_label = ttk.Label(
    root,
    text="date",
    width=5,
    relief=tk.SUNKEN,
)
date_label.grid(row=2, column=0, padx=20, pady=5)




calender_date = Calendar(root,background="black", disabledbackground="black", bordercolor="black", 
               headersbackground="black", normalbackground="black", foreground='white', 
               normalforeground='white', headersforeground='white',showweeknumbers=False,date_patternstr = 'y-mm-dd')
calender_date.grid(row=3, column=1)

date = ttk.Label(
    root,
    text=calender_date.get_date(),
    width=5,
    relief=tk.SUNKEN,
)
date.grid(row=2, column=1, padx=20, pady=5)



draw = tk.StringVar(root)
draw.set("DRAW!")
draw_button = tk.Button(root, textvariable=draw, command=draw_graph)
draw_button.grid(row=4, column=1, padx=10, pady=5)

root.mainloop()